# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770797255293                   -0.53    8.0    185ms
  2   -2.772140796864       -2.87       -1.31    1.0    104ms
  3   -2.772170392297       -4.53       -2.55    1.0    123ms
  4   -2.772170690995       -6.52       -3.51    1.0    146ms
  5   -2.772170721212       -7.52       -3.91    2.0    123ms
  6   -2.772170722973       -8.75       -5.46    1.0    110ms
  7   -2.772170723014      -10.39       -5.46    3.0    156ms
  8   -2.772170723015      -11.82       -6.36    1.0    114ms
  9   -2.772170723015      -13.94       -6.80    1.0    130ms
 10   -2.772170723015      -14.88       -7.66    1.0    116ms
 11   -2.772170723015      -14.18       -7.82    2.0    148ms
 12   -2.772170723015   +  -14.40       -8.62    1.0    126ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.773557909182737

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770767735584                   -0.52    9.0    209ms
  2   -2.772060329011       -2.89       -1.32    1.0    105ms
  3   -2.772083005592       -4.64       -2.43    1.0    106ms
  4   -2.772083332396       -6.49       -3.12    1.0    148ms
  5   -2.772083417199       -7.07       -4.10    2.0    124ms
  6   -2.772083417698       -9.30       -4.63    1.0    111ms
  7   -2.772083417806       -9.96       -5.72    1.0    131ms
  8   -2.772083417810      -11.41       -5.90    2.0    127ms
  9   -2.772083417811      -12.54       -7.17    1.0    134ms
 10   -2.772083417811   +  -14.57       -7.57    2.0    137ms
 11   -2.772083417811      -15.35       -8.11    2.0    140ms

Polarizability via ForwardDiff:       1.772534966639569
Polarizability via finite difference: 1.773557909182737
